In [1]:
!pip install nest_asyncio==1.3.3

In [2]:
!pip install rasa_nlu[spacy]

In [3]:
!pip install rasa_core

In [4]:
!pip install rasa-sdk

In [5]:
pip install -U ipython 

In [6]:
import rasa_core
import rasa_nlu
import spacy
import sys
python = sys.executable

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got

In [7]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 120.8 MB 11.3 MB/s 

    Linking successful
    /usr/local/lib/python3.7/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.7/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [8]:
# Writing various intents with examples to nlu.md file
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- how far
- hey there
- whats up
- hey dude
- goodmorning
- goodevening
- good afternoon
## intent:goodbye
- good by
- later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later
## intent:thanks
- thanks
- thank you
- appreciated
- gracias
## intent:Check Capital
- what is the capital of [USA](country)
- I need to know the capital of [Russia](country)
- can you tell me the capital of [Japan](country)
- do you know the capital of [Finland](country)
- what is the main city of [Australia](country)
- tell me the capital of [Sweden](country)
- I wounder what is the capital of [Greece](country)
- I want to know the capital of [Egypt](country)
- please, tell me the capital of [India](country)
## intent:Check Population
- what is the population of [USA](country)
- I need to know the population of [Japan](country)
- can you tell me the population of [Egypt](country)
- do you know the population of [Sweden](country)
- How many people live in [Greece](country)
- tell me the population of [India](country)
- I wounder what is the population of [Finland](country)
- I want to know the population of [Australia](country)
- please, tell me the population of [Russia](country)
## intent:choosing_country
- 1
- 2
- 3
- 4
- 5
- 6
- 7
- 8
"""
%store nlu_md >nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [9]:
# Adding the NLU components to the pipeline in config.yml file
config = """
language: "en_core_web_md"
pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                     # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms
policies:
- name: "RulePolicy"
# a confidence >= core_fallback_threshold
core_fallback_threshold: 0.68                            # Confidence threshold for the `core_fallback_action_name` to apply.
core_fallback_action_name: "action_default_fallback"     # The action will apply if no other action was predicted with
enable_fallback_prediction: True                         # a confidence >= core_fallback_threshold
"""
%store config >config.yml

Writing 'config' (str) to file 'config.yml'.


In [10]:
# Import modules for training
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config
# loading the nlu training samples
training_data = load_data("nlu.md")
trainer = Trainer(config.load("config.yml"))
# training the nlu
interpreter = trainer.train(training_data)
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_

In [11]:
# Testing the NLU model with an input message
import json
def pprint(o):
    print(json.dumps(o, indent=2))
pprint(interpreter.parse("what is the capital of Germany"))

{
  "intent": {
    "name": "Check Capital",
    "confidence": 0.8767496201017753
  },
  "entities": [
    {
      "start": 23,
      "end": 30,
      "value": "germany",
      "entity": "country",
      "confidence": 0.8335545992672485,
      "extractor": "CRFEntityExtractor"
    }
  ],
  "intent_ranking": [
    {
      "name": "Check Capital",
      "confidence": 0.8767496201017753
    },
    {
      "name": "greet",
      "confidence": 0.03544658418569139
    },
    {
      "name": "goodbye",
      "confidence": 0.028221707322129124
    },
    {
      "name": "Check Population",
      "confidence": 0.024437467415179652
    },
    {
      "name": "choosing_country",
      "confidence": 0.02113584512742315
    },
    {
      "name": "thanks",
      "confidence": 0.014008775847801646
    }
  ],
  "text": "what is the capital of Germany"
}


In [12]:
# Writing stories and saving it in the stories.md file
stories_md = """
##  Greeting path
* greet
- utter_greet
## capital path
* choosing_country
- utter_country
* Check Capital
- utter_capital
- action_get_capital
## Population path
* choosing_country
- utter_country
* Check Population
- utter_population
## say goodbye
* goodbye
- utter_goodbye
## thanks
* thanks
- utter_thanks
## fallback rule
* bot_fallback
- action_default_fallback
"""
%store stories_md >stories.md

Writing 'stories_md' (str) to file 'stories.md'.


In [13]:
# Writing all the intents,slots,entities,actions and templates to domain.yml
domain_yml = """
intents:
- greet
- goodbye
- Check Capital
- Check Population
- thanks
slots:
  country:
    type: text
entities:
- country
actions:
- utter_greet
- utter_country
- utter_capital
- utter_population
- utter_goodbye
- utter_thanks
- utter_default
- action_get_capital
templates:
  utter_greet:
  - text: "\n\n BOT: \n\n Hey! How may I help you ? \n\n \n\n You:"
  utter_country:
  - text: "\n\n BOT: \n\n Please choose the country:  \n\n 1. USA  2. Greece 3. Sweden  4. Australia  5. Finland  6. Japan  7. Russia  8. India  \n\n Enter the number of the country you want: \n\n \n\n  You:"
  utter_capital:
  - text: "\n\n BOT: \n\n The capital of [country] is . \n\n \n\n You:"
  utter_population:
  - text: "\n\n BOT: \n\n The population of [country] is . \n\n \n\n You:"
  utter_goodbye:
  - text: "\n\n BOT: \n\n Chatting with you was nice. Waiting for you again. \n\n \n\n You:"
  utter_thanks:
  - text: "\n\n BOT: \n\n my pleasure \n\n \n\n You:"
  utter_default:
  - text: "\n\n BOT: \n\n I'm sorry, I didn't quite understand that. Could you rephrase? \n\n You:"
"""
%store domain_yml >domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


Adding Custom Actions:

With an API is created a custom action that affords the chatbot to retrieve an answer for the country entity capital or population depending on the user inputs. The chatbot will know which type of answer should be given by retrieving the value from the slot 'group'.

In [14]:
# from rasa_core.actions import Action
# from rasa_core.events import SlotSet
# from IPython.core.display import Image, display

# import requests

# class ApiAction(Action):
#     def name(self):
#         return "action_get_info"

#     def run(self, dispatcher, tracker, domain):
        
#         group = tracker.get_slot('Ambulance','Insurance','Fire','Police','Air_rescue','Road_help')
        
#         r = requests.get('http://shibe.online/api/{}?count=1&urls=true&httpsUrls=true'.format(group))
#         response = r.content.decode()
#         response = response.replace('["',"")
#         response = response.replace('"]',"")


In [15]:
from typing import Any, Text, Dict, List

from rasa_sdk import Action, Tracker
from rasa_sdk.events import SlotSet
from rasa_sdk.executor import CollectingDispatcher
import requests


class ActionCheckCapital(Action):

    def name(self)-> Text:
        return "action_get_capital"
    
    def run(self, dispatcher, tracker, domain):
        api_key = 'Your API Key'
        loc = tracker.get_slot('country')
        current = requests.get('https://restcountries.com/v3.1/name/{}?fullText=true'.format(loc)).json()
        print(current)
        name= current['name']['official']
        capital = current['capital']
        population = current['population']
        response = """The Capital of {} is {} and population is {} """.format(name, capital, population)
        dispatcher.utter_message(response)
        return [SlotSet('country', loc)]


In [16]:
# Import the policies and agent
from rasa_core.policies import FallbackPolicy, MemoizationPolicy,KerasPolicy
from rasa_core.agent import Agent
# Initialize the model with `domain.yml`
agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), ])
# loading our  training dialogues from `stories.md`
training_data = agent.load_data('stories.md')
# Training the model
agent.train(
training_data)
#validation_split=0.0,
#epochs=200)
agent.persist('models/dialogue')

Processed Story Blocks: 100%|██████████| 6/6 [00:00<00:00, 66.63it/s, # trackers=16]
Processed actions: 16it [00:00, 555.76it/s, # examples=16]


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 23)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7168      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
activation (Activation)      (None, 16)                0         
Total params: 7,696
Trainable params: 7,696
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
280/280 [================

In [17]:
!pip uninstall h5py
!pip install h5py==2.10.0

Found existing installation: h5py 2.10.0
Uninstalling h5py-2.10.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/h5py-2.10.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-2.10.0
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)


In [18]:
#Starting the Bot
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [19]:
print("Your bot is ready to talk! Type your messages here or send 'stop'" + "\n\n" + "You:")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])


Your bot is ready to talk! Type your messages here or send 'stop'

You:
hello

BOT:
Hey! How may I help you ?

You:
good morning

BOT:
Hey! How may I help you ?

You:
tell me the capital of france

BOT:
I'm sorry, I didn't quite understand that. Could you rephrase?
You:
what is the population of USA

BOT:
Please choose the country:
1. USA  2. Greece 3. Sweden  4. Australia  5. Finland  6. Japan  7. Russia  8. India
Enter the number of the country you want:

You:
1

BOT:
The population of [country] is .

You:
what is the capital of USA

BOT:
The capital of [country] is .

You:
thank you

BOT:
my pleasure

You:
bye

BOT:
Chatting with you was nice. Waiting for you again.

You:
stop


In [20]:
!zip -r models.zip models/

updating: models/ (stored 0%)
updating: models/dialogue/ (stored 0%)
updating: models/dialogue/policy_0_MemoizationPolicy/ (stored 0%)
updating: models/dialogue/policy_0_MemoizationPolicy/featurizer.json (deflated 43%)
updating: models/dialogue/policy_0_MemoizationPolicy/memorized_turns.json (deflated 50%)
updating: models/dialogue/policy_1_KerasPolicy/ (stored 0%)
updating: models/dialogue/policy_1_KerasPolicy/featurizer.json (deflated 59%)
updating: models/dialogue/policy_1_KerasPolicy/keras_model.h5 (deflated 32%)
updating: models/dialogue/policy_1_KerasPolicy/keras_policy.json (deflated 15%)
updating: models/dialogue/policy_1_KerasPolicy/keras_policy.tf_config.pkl (stored 0%)
updating: models/dialogue/metadata.json (deflated 60%)
updating: models/dialogue/domain.yml (deflated 57%)
updating: models/dialogue/domain.json (deflated 67%)
updating: models/nlu/ (stored 0%)
updating: models/nlu/default/ (stored 0%)
updating: models/nlu/default/current/ (stored 0%)
updating: models/nlu/defa